# データベース
ここではデータベースからデータを取り出す際に利用するSQLの基本について解説する。

#### 注意
Windowsを利用している場合はこちらを参考にSqliteのインストールとパスを通すことが事前に必要となります。  
難しい場合は、一通り目を通してSQLの使い方を把握するだけでも構いません。（SQLについてはこの後のセクションで必要になることはありません。）

ダウンロード： https://www.sqlite.org/download.html

In [1]:
# pandasql のインストール（インストールしていない場合は）
!pip install pandasql

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
from pandasql import sqldf

In [3]:
# データの読み込み 4章で利用したMovie Lensを利用
# ユーザー情報
user_url = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.user'
users = pd.read_csv(user_url, sep='|', header=None, encoding='latin-1')
user_col_info = "user id | age | gender | occupation | zip code"
users.columns = user_col_info.split(" | ")

# 評価情報
rating_url = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'
ratings = pd.read_csv(rating_url, sep='\t', header=None, encoding='latin-1')
rating_col_info = "user id | item id | rating | timestamp"
ratings.columns = rating_col_info.split(" | ")

## データベースの基本用語
- データベース ... データベース（アプリケーション）の中に作られる表データのまとまり
- テーブル ... 個々の表データ（pandasのデータフレームに相当）
- カラム ... テーブルの各列

## これ以降のSQL利用イメージ
下記のようなイメージを持ってSQL文を作ってみる（実際に以下のようなシーンは実務で多々ある）

- データベースにアクセスしている
- 以下2つのテーブルが存在
  - users
  - rating
- SQL文はデータベースに対して発行している
- 結果をpandasのDataFrameとして受け取り

## SQLの基本構文
### select文
- select ... 対象の列を指定する（`*`は全てという意味）
- from ... 対象のテーブルを指定する

In [4]:
# 結果確認用に取得した結果の全体像とTOP5行を返す関数
def get_from_sql(query):
    return_df = sqldf(query)
    print("shape: ", return_df.shape)
    return return_df.head()

In [5]:
# select文
query = "select * from users"
get_from_sql(query)

shape:  (943, 5)


,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [6]:
# select文 カラム選択
query = "select `user id`, age from users"
get_from_sql(query)

shape:  (943, 2)


,user id,age
0,1,24
1,2,53
2,3,23
3,4,24
4,5,33


In [7]:
# select文 カラム選択 limit節で取得データの行数制限
query = "select `user id`, age from users limit 1"
get_from_sql(query)

shape:  (1, 2)


,user id,age
0,1,24


### where節による絞り込み
`where` の後ろに条件を記述することで特定の行を絞り込むことができる

In [8]:
# 20歳のユーザーに限定
# SQLは複数行で記述することもできる
query = """
select
  *
from
  users
where
  age = 20
"""
get_from_sql(query)

shape:  (32, 5)


,user id,age,gender,occupation,zip code
0,35,20,F,homemaker,42459
1,76,20,M,student,02215
2,99,20,M,student,63129
3,117,20,M,student,16125
4,130,20,M,none,60115


In [9]:
# 20歳で男性のユーザーに限定（複数条件）
query = """
select
  *
from
  users
where
  age = 20 and gender = "M"
"""
get_from_sql(query)

shape:  (22, 5)


,user id,age,gender,occupation,zip code
0,76,20,M,student,02215
1,99,20,M,student,63129
2,117,20,M,student,16125
3,130,20,M,none,60115
4,139,20,M,student,08904


### order by節
`order by` を用いることで並び替えを行うことができる。

In [10]:
# 学生を年齢順に並び替える（降順はdesc、昇順はasc）
query = """
select
  *
from
  users
where
  occupation = "student"
order by age asc
"""
get_from_sql(query)

shape:  (196, 5)


,user id,age,gender,occupation,zip code
0,30,7,M,student,55436
1,471,10,M,student,77459
2,609,13,F,student,55106
3,674,13,F,student,55337
4,880,13,M,student,83702


## 集約関数
### よく使われる集約関数
- `count(col or *)` ... colの行数を数える/もしくは全ての行数を数える
- `sum(col)` ... colの合計
- `avg(col)` ... colの平均
- `min(col)` ... colの最小値
- `max(col)` ... colの最大値

In [11]:
# 行数を数える
query = "select count(*) from users"
get_from_sql(query)

shape:  (1, 1)


,count(*)
0,943


In [12]:
# 年齢の平均を求める
query = "select avg(age) from users"
get_from_sql(query)

shape:  (1, 1)


,avg(age)
0,34.051962


### group by節による対象ごとの集約値
`group by` の後に特定のカラム名を指定すると、そのカラムの値別に集約値を求めることができる。

In [13]:
# 職業別に平均の年齢を求める
#   as を使うと列名を変更できる
query = """
select
  occupation, avg(age) as avg_age
from
  users
group by
  occupation
order by avg_age asc
"""
get_from_sql(query)

shape:  (21, 2)


,occupation,avg_age
0,student,22.081633
1,none,26.555556
2,entertainment,29.222222
3,artist,31.392857
4,homemaker,32.571429


In [14]:
# having節を用いると、集計後の値で絞り込むことができる
query = """
select
  occupation, avg(age) as avg_age
from
  users
group by
  occupation
having
  avg_age < 30
order by avg_age asc
"""
get_from_sql(query)

shape:  (3, 2)


,occupation,avg_age
0,student,22.081633
1,none,26.555556
2,entertainment,29.222222


In [15]:
# group by節で2つ以上のカラムを指定することもできる
#   職業、性別ごとの平均年齢を取得
#   order byも2つ以上カラムを指定することができる
query = """
select
  occupation, gender, avg(age) as avg_age
from
  users
group by
  occupation, gender
order by
 occupation, gender
"""
get_from_sql(query)

shape:  (41, 3)


,occupation,gender,avg_age
0,administrator,F,40.638889
1,administrator,M,37.162791
2,artist,F,30.307692
3,artist,M,32.333333
4,doctor,M,43.571429


## テーブルの結合（join句）
先ほどのセクションで確認した4つの結合がSQLでもできる。（正確にはSQLで利用されているテーブルの結合がpandasでもできる）

 right join と outer joinはライブラリで未対応なためここではスキップする。


In [16]:
# inner join
query = """
select
  *
from
  users
  inner join ratings
  on users.`user id` = ratings.`user id`
"""
get_from_sql(query)

shape:  (100000, 9)


,user id,age,gender,occupation,zip code,user id,item id,rating,timestamp
0,1,24,M,technician,85711,1,1,5,874965758
1,1,24,M,technician,85711,1,2,3,876893171
2,1,24,M,technician,85711,1,3,4,878542960
3,1,24,M,technician,85711,1,4,3,876893119
4,1,24,M,technician,85711,1,5,3,889751712


In [17]:
# left join
query = """
select
  *
from
  users
  left join ratings
  on users.`user id` = ratings.`user id`
"""
get_from_sql(query)

shape:  (100000, 9)


,user id,age,gender,occupation,zip code,user id,item id,rating,timestamp
0,1,24,M,technician,85711,1,1,5,874965758
1,1,24,M,technician,85711,1,2,3,876893171
2,1,24,M,technician,85711,1,3,4,878542960
3,1,24,M,technician,85711,1,4,3,876893119
4,1,24,M,technician,85711,1,5,3,889751712


## その他知っておくと良いこと
- 種々の関数（文字列や日付の操作など）
- サブクエリ（SQLで得られるテーブルに対する操作）
- ウィンドウ関数（より高度な分析）

## おすすめ書籍
青木峰郎著『10年戦えるデータ分析入門　SQLを武器にデータ活用時代を生き抜く』

## 参考資料
- PostgreSQLドキュメント（SQLやデータベースについて知れる） ... https://www.postgresql.jp/document/12/html/
- BigQuery文法 ... https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax?hl=ja